In [7]:
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["axes.unicode_minus"] = False
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as mcolors

from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter

from datetime import datetime
from zoneinfo import ZoneInfo

# ===============================================================================================================================
# 
# SCU SOCCER ATHLETE MONITORING PIPELINE USING CATAPULT & VALD DATA
# 
# REQUIRED INPUT FILES (same folder as this script):
#   - catapult_profiles.csv
#   - catapult_report.csv
#   - forcedecks_profiles.csv
#   - nordbord_profiles.csv
#
# EXPECTED OUTPUT FILES (same folder as this script):
#   - STEP 1) complete_data.csv
#   - STEP 2) reports/metric_visualizations.pdf
#   - STEP 3) reports/player_visualizations.pdf
#   - STEP 4) reports/metric_spreadsheets.xlsx
#   - STEP 5) reports/player_spreadsheets.xlsx
# 
# ===============================================================================================================================

# ===============================================================
# CONFIGURE THESE BEFORE RUNNING (SHIFT ENTER TO RUN IPYNB FILE)
# ===============================================================

#(1) Must have the following four files in the same folder to work
CATAPULT_PROFILES_CSV = "catapult_profiles.csv"
CATAPULT_REPORT_CSV = "catapult_report.csv"
FORCEDECKS_CSV = "forcedecks_profiles.csv"
NORDBORD_CSV = "nordbord_profiles.csv"

#(2) File name of the cleaned file that generates in step 1
COMPLETE_CSV = "complete_data.csv"

#(3) Specific metrics you might want to exclude for steps 2 to 5
EXCLUDED_METRICS = [
    "red_zone",
    "avg_trimp",
    "percentage_max_heart_rate",
    "Peak Power / BM",
    "Countermovement Depth",
]

#(4) Specific players you might want to exclude for steps 2 to 5
EXCLUDED_PLAYERS = [
    # "Player 1",
]

#(5) Steps you want to run or recompile
RUN_COMPLETE_DATA = True #STEP 1
RUN_METRIC_VISUALIZATIONS = True #STEP 2
RUN_PLAYER_VISUALIZATIONS = True #STEP 3
RUN_METRIC_SPREADSHEETS = True #STEP 4
RUN_PLAYER_SPREADSHEETS = True #STEP 5


#(6) Categorize remaining metrics by catapult, forcedecks, and nordbord for steps 2 to 5
CATAPULT_METRICS = [
    ("total_distance", "Total Distance"),  #("actual name in csv", "name you want to show")
    ("high_speed_distance", "High Speed Distance"),
    ("percentage_max_velocity", "% Max Velocity"),
    ("gen2_acceleration_band3plus_total_effort_count", "Band 3+ Acceleration Efforts"),
    ("gen2_acceleration_band6plus_total_effort_count", "Band 6+ Acceleration Efforts"),
    ("total_player_load", "Player Load"),
    ("player_load_per_minute", "Player Load Per Minute"),
    ("meterage_per_minute", "Meterage Per Minute"),
    ("total_acceleration_load", "Acceleration Load"),
]

FORCEDECKS_METRICS = [
    ("Jump Height (Flight Time)", "Jump Height (Flight Time)"),
    ("RSI-Modified", "RSI (Modified)"),
    ("Concentric Mean Force", "Concentric Mean Force"),
]

NORDBORD_PAIRS = [
    ("AvgForce", "Average Force"),
    ("Impulse", "Impulse"),
    ("MaxForce", "Max Force"),
    ("Torque", "Torque"),
]

#(7) |% change| needed to be considered an outlier
OUTLIER_THRESHOLD = 20

#(8) Customize color scheme
BG_COLOR = "#0e0f12"
FG_COLOR = "#e8e8e8"
MUTED = "#a0a0a0"
GRID_COLOR = "#3a3a3a"
ACCENT = "#59a14f" #within threshold
RISK = "#e15759" #outside threshold

# ============================================================
# REST OF THE CODE (DO NOT TOUCH)
# ============================================================






def _make_generated_ts(tz_name="America/Los_Angeles"):
    """Returns a display string like 'Generated: December 16, 2025 2:30 PM PST'."""
    try:
        dt = datetime.now(ZoneInfo(tz_name))
    except Exception:
        dt = datetime.now()
    s = dt.strftime("%B %d, %Y %I:%M %p %Z").replace(" 0", " ")
    return f"Generated: {s}".strip()

def _add_text_page(pdf, title, subtitle=None, lines=None):
    """Adds a simple text-only page to an existing PdfPages object."""
    fig = plt.figure(figsize=(16, 9), facecolor=BG_COLOR)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_facecolor(BG_COLOR)
    ax.axis("off")

    ax.text(
        0.5, 0.62, title,
        ha="center", va="center",
        fontsize=46, fontweight="bold",
        color=FG_COLOR
    )

    if subtitle:
        ax.text(
            0.5, 0.48, subtitle,
            ha="center", va="center",
            fontsize=20,
            color=MUTED
        )

    if lines:
        y = 0.36
        for line in lines:
            ax.text(
                0.10, y, line,
                ha="left", va="center",
                fontsize=20,
                color=MUTED
            )
            y -= 0.07

    pdf.savefig(fig, facecolor=BG_COLOR)
    plt.close(fig)

def _add_how_to_page(pdf, bullets):
    fig = plt.figure(figsize=(16, 9), facecolor=BG_COLOR)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_facecolor(BG_COLOR)
    ax.axis("off")

    ax.text(
        0.10, 0.78, "HOW TO USE:",
        ha="left", va="center",
        fontsize=44, fontweight="bold",
        color=FG_COLOR
    )

    y = 0.60
    for b in bullets:
        ax.text(
            0.12, y, f"- {b}",
            ha="left", va="center",
            fontsize=22,
            color=MUTED
        )
        y -= 0.10

    pdf.savefig(fig, facecolor=BG_COLOR)
    plt.close(fig)

def style_axis(ax):
    ax.set_facecolor(BG_COLOR)
    for s in ax.spines.values():
        s.set_color(GRID_COLOR)
    ax.tick_params(colors=FG_COLOR, labelcolor=FG_COLOR)
    ax.xaxis.label.set_color(FG_COLOR)
    ax.yaxis.label.set_color(FG_COLOR)
    ax.title.set_color(FG_COLOR)

def fmt_num(x):
    if pd.isna(x):
        return "-"
    ax = abs(float(x))
    if ax >= 1000:
        return f"{x:,.0f}"
    if ax >= 100:
        return f"{x:,.1f}"
    if ax >= 10:
        return f"{x:,.2f}"
    return f"{x:,.3f}"

def is_valid_nonzero(x):
    return (pd.notna(x) and np.isfinite(x) and float(x) != 0.0)

def safe_ratio(recent, avg):
    if pd.isna(recent) or pd.isna(avg) or abs(float(avg)) < 1e-9:
        return np.nan
    r = float(recent) / float(avg)
    return np.nan if not np.isfinite(r) else float(r)

def pct_from_ratio(r):
    return np.nan if pd.isna(r) else (float(r) - 1.0) * 100.0

def calc_asym(L, R):
    if pd.isna(L) or pd.isna(R) or abs(float(L) + float(R)) < 1e-9:
        return np.nan
    return (float(R) - float(L)) / ((float(R) + float(L)) / 2.0) * 100.0

def build_complete_data_csv(
    catapult_profiles_path=CATAPULT_PROFILES_CSV,
    catapult_metrics_path=CATAPULT_REPORT_CSV,
    forcedecks_path=FORCEDECKS_CSV,
    nordbord_path=NORDBORD_CSV,
    output_csv=COMPLETE_CSV,
    excluded_metrics=None,
    excluded_players=None,
):
    excluded_metrics = excluded_metrics or []
    excluded_players = excluded_players or []

    cat_profiles = pd.read_csv(catapult_profiles_path)
    cat_metrics  = pd.read_csv(catapult_metrics_path)

    cat_id_cols = ["player_id", "player_name"]

    cat_profiles_extra = [c for c in cat_profiles.columns if c not in cat_id_cols]
    cat_metrics_extra  = [c for c in cat_metrics.columns  if c not in cat_id_cols]

    cat_profiles_renamed = cat_profiles.rename(
        columns={
            c: (c + "_season" if c != "num_periods" else "num_periods_season")
            for c in cat_profiles_extra
        }
    )
    cat_metrics_renamed = cat_metrics.rename(
        columns={c: c + "_period" for c in cat_metrics_extra}
    )

    catapult_merged = pd.merge(
        cat_profiles_renamed,
        cat_metrics_renamed,
        on=cat_id_cols,
        how="outer"
    )

    forcedecks = pd.read_csv(forcedecks_path)
    nordbord   = pd.read_csv(nordbord_path)

    vald_id_cols = ["player_id", "player_name", "vald_id"]

    vald_merged = pd.merge(
        forcedecks,
        nordbord,
        on=vald_id_cols,
        how="outer"
    )

    if "player_id" in catapult_merged.columns:
        catapult_merged = catapult_merged.rename(columns={"player_id": "player_id_catapult"})
    if "player_id" in vald_merged.columns:
        vald_merged = vald_merged.rename(columns={"player_id": "player_id_vald"})

    complete = pd.merge(
        catapult_merged,
        vald_merged,
        on="player_name",
        how="outer"
    )

    cols_to_drop = [c for c in complete.columns if "_id" in c]
    if cols_to_drop:
        complete = complete.drop(columns=cols_to_drop)

    if excluded_players:
        complete = complete[~complete["player_name"].isin(excluded_players)].copy()

    season_suffix = "_season"
    period_suffix = "_period"
    n_col = "num_periods_season"

    catapult_bases = []
    for col in complete.columns:
        if col.endswith(season_suffix) and col != n_col:
            base = col[:-len(season_suffix)]
            if base not in catapult_bases:
                catapult_bases.append(base)

    for base in catapult_bases:
        if base in excluded_metrics:
            continue
        season_col = base + season_suffix
        period_col = base + period_suffix

        if season_col in complete.columns:
            complete[base + "_avg"] = complete[season_col]
        if period_col in complete.columns:
            complete[base + "_recent"] = complete[period_col]

    drop_cols = [n_col] if n_col in complete.columns else []
    for base in catapult_bases:
        sc = base + season_suffix
        pc = base + period_suffix
        if sc in complete.columns:
            drop_cols.append(sc)
        if pc in complete.columns:
            drop_cols.append(pc)

    if drop_cols:
        complete = complete.drop(columns=drop_cols)

    vald_bases = []
    for col in complete.columns:
        if col.endswith("_avg"):
            base = col[:-4]
            if (
                base not in catapult_bases and
                base not in vald_bases and
                base not in excluded_metrics
            ):
                vald_bases.append(base)

    ordered_cols = ["player_name"]
    for base in catapult_bases + vald_bases:
        if base in excluded_metrics:
            continue
        avg_col = base + "_avg"
        rec_col = base + "_recent"
        if avg_col in complete.columns:
            ordered_cols.append(avg_col)
        if rec_col in complete.columns:
            ordered_cols.append(rec_col)

    complete = complete[ordered_cols]
    complete.to_csv(output_csv, index=False)
    return complete

def build_long_outlier_table(
    df,
    pct_band=OUTLIER_THRESHOLD,
    asym_band=OUTLIER_THRESHOLD,
    treat_zero_as_missing=True,
):
    def valid(x):
        if pd.isna(x) or not np.isfinite(x):
            return False
        if treat_zero_as_missing and float(x) == 0.0:
            return False
        return True

    def safe_ratio_team(recent, baseline):
        if (not valid(recent)) or (not valid(baseline)) or abs(float(baseline)) < 1e-9:
            return np.nan
        r = float(recent) / float(baseline)
        return r if np.isfinite(r) else np.nan

    def pct_from_ratio_team(r):
        return np.nan if pd.isna(r) else (float(r) - 1.0) * 100.0

    def calc_asym_team(L, R):
        if pd.isna(L) or pd.isna(R) or abs(float(L) + float(R)) < 1e-9:
            return np.nan
        return (float(R) - float(L)) / ((float(R) + float(L)) / 2.0) * 100.0

    rows = []

    for base, label in CATAPULT_METRICS:
        bcol = f"{base}_avg"
        rcol = f"{base}_recent"
        if bcol not in df.columns or rcol not in df.columns:
            continue
        for _, row in df.iterrows():
            player = row.get("player_name", "Unknown")
            baseline = row.get(bcol, np.nan)
            recent   = row.get(rcol, np.nan)
            pct = pct_from_ratio_team(safe_ratio_team(recent, baseline))
            is_out = (np.isfinite(pct) and (abs(pct) > pct_band))
            rows.append({"group":"CATAPULT","metric":label,"player":player,"x":baseline,"y":recent,"pct":pct,"is_outlier":bool(is_out)})

    for base, label in FORCEDECKS_METRICS:
        bcol = f"{base}_avg"
        rcol = f"{base}_recent"
        if bcol not in df.columns or rcol not in df.columns:
            continue
        for _, row in df.iterrows():
            player = row.get("player_name", "Unknown")
            baseline = row.get(bcol, np.nan)
            recent   = row.get(rcol, np.nan)
            pct = pct_from_ratio_team(safe_ratio_team(recent, baseline))
            is_out = (np.isfinite(pct) and (abs(pct) > pct_band))
            rows.append({"group":"FORCEDECKS","metric":label,"player":player,"x":baseline,"y":recent,"pct":pct,"is_outlier":bool(is_out)})

    for suf, label in NORDBORD_PAIRS:
        for side in ("Left", "Right"):
            bcol = f"{side.lower()}{suf}_avg"
            rcol = f"{side.lower()}{suf}_recent"
            if bcol not in df.columns or rcol not in df.columns:
                continue
            metric_label = f"{side} {label}"
            for _, row in df.iterrows():
                player = row.get("player_name", "Unknown")
                baseline = row.get(bcol, np.nan)
                recent   = row.get(rcol, np.nan)
                pct = pct_from_ratio_team(safe_ratio_team(recent, baseline))
                is_out = (np.isfinite(pct) and (abs(pct) > pct_band))
                rows.append({"group":"NORDBORD","metric":metric_label,"player":player,"x":baseline,"y":recent,"pct":pct,"is_outlier":bool(is_out)})

    for suf, label in NORDBORD_PAIRS:
        lrec = f"left{suf}_recent"; rrec = f"right{suf}_recent"
        if (lrec in df.columns) and (rrec in df.columns):
            for _, row in df.iterrows():
                player = row.get("player_name", "Unknown")
                L = row.get(lrec, np.nan)
                R = row.get(rrec, np.nan)
                a = calc_asym_team(L, R)
                is_out = (np.isfinite(a) and (abs(a) > asym_band))
                rows.append({"group":"NORDBORD ASYMMETRY (RECENT)","metric":label,"player":player,"x":R,"y":L,"pct":a,"is_outlier":bool(is_out)})

        lavg = f"left{suf}_avg"; ravg = f"right{suf}_avg"
        if (lavg in df.columns) and (ravg in df.columns):
            for _, row in df.iterrows():
                player = row.get("player_name", "Unknown")
                L = row.get(lavg, np.nan)
                R = row.get(ravg, np.nan)
                a = calc_asym_team(L, R)
                is_out = (np.isfinite(a) and (abs(a) > asym_band))
                rows.append({"group":"NORDBORD ASYMMETRY (AVG)","metric":label,"player":player,"x":R,"y":L,"pct":a,"is_outlier":bool(is_out)})

    long = pd.DataFrame(rows)
    if long.empty:
        return long

    def _row_has_any_data(r):
        return valid(r["x"]) or valid(r["y"])
    long = long[long.apply(_row_has_any_data, axis=1)].copy()
    return long

def band_for_group(group, pct_band, asym_band):
    return asym_band if str(group).startswith("NORDBORD ASYMMETRY") else pct_band

def plot_player_rankings_page(long, pdf=None, top_n=10,
                              pct_band: float = OUTLIER_THRESHOLD, asym_band: float = OUTLIER_THRESHOLD):

    if long.empty:
        return

    _m_out, _m_opt, player_out, player_opt = _build_rank_tables(long, pct_band, asym_band)
    if player_out.empty and player_opt.empty:
        return

    out_counts = (
        player_out.head(top_n)
        .set_index("player_name")["outlier_metrics"]
        .astype(float)
        if not player_out.empty else pd.Series(dtype=float)
    )
    opt_counts = (
        player_opt.head(top_n)
        .set_index("player_name")["optimal_metrics"]
        .astype(float)
        if not player_opt.empty else pd.Series(dtype=float)
    )

    fig, axes = plt.subplots(1, 2, figsize=(16, 7.5), facecolor=BG_COLOR)
    ax1, ax2 = axes

    style_axis(ax1)
    if not out_counts.empty:
        ax1.barh(out_counts.index.tolist(), out_counts.values.tolist(), color=mcolors.to_rgba(RISK, 0.9))
        ax1.invert_yaxis()
        ax1.set_title(f"Top {len(out_counts)} Players: Most Outlier Metrics", fontsize=14, pad=12)
        ax1.set_xlabel("Outlier metric count", fontsize=11)
        ax1.grid(axis="x", color=GRID_COLOR, alpha=0.45)
        for y, v in enumerate(out_counts.values.tolist()):
            ax1.text(v + 0.15, y, f"{int(v)}", va="center", color=FG_COLOR, fontsize=10)
    else:
        ax1.axis("off")

    style_axis(ax2)
    if not opt_counts.empty:
        ax2.barh(opt_counts.index.tolist(), opt_counts.values.tolist(), color=mcolors.to_rgba(ACCENT, 0.9))
        ax2.invert_yaxis()
        ax2.set_title(f"Top {len(opt_counts)} Players: Most Optimal Metrics", fontsize=14, pad=12)
        ax2.set_xlabel("Optimal metric count", fontsize=11)
        ax2.grid(axis="x", color=GRID_COLOR, alpha=0.45)
        for y, v in enumerate(opt_counts.values.tolist()):
            ax2.text(v + 0.15, y, f"{int(v)}", va="center", color=FG_COLOR, fontsize=10)
    else:
        ax2.axis("off")

    fig.suptitle("PLAYER RANKINGS", y=0.98, fontsize=18, color=FG_COLOR)
    fig.tight_layout(rect=[0, 0, 1, 0.94])

    if pdf is not None:
        pdf.savefig(fig, facecolor=BG_COLOR)
        plt.close(fig)
    else:
        plt.show()

def plot_metric_rankings_page(long, pdf=None, top_n=12,
                              pct_band: float = OUTLIER_THRESHOLD, asym_band: float = OUTLIER_THRESHOLD):

    if long.empty:
        return

    metric_out, _metric_opt, _player_out, _player_opt = _build_rank_tables(long, pct_band, asym_band)
    if metric_out.empty:
        return

    top = metric_out.head(top_n).copy()
    labels = (top["group"].astype(str) + " | " + top["metric"].astype(str)).tolist()
    counts = top["outlier_players"].astype(float).tolist()

    fig, ax = plt.subplots(1, 1, figsize=(16, 8), facecolor=BG_COLOR)
    style_axis(ax)

    ax.barh(labels, counts, color=mcolors.to_rgba(RISK, 0.9))
    ax.invert_yaxis()
    ax.set_title(f"Top {len(labels)} Metrics: Most Outlier Players", fontsize=16, pad=12)
    ax.set_xlabel("Outlier player count", fontsize=12)
    ax.grid(axis="x", color=GRID_COLOR, alpha=0.45)

    for y, v in enumerate(counts):
        ax.text(v + 0.15, y, f"{int(v)}", va="center", color=FG_COLOR, fontsize=10)

    fig.suptitle("METRIC RANKINGS", y=0.98, fontsize=20, color=FG_COLOR)
    fig.tight_layout(rect=[0, 0, 1, 0.94])

    if pdf is not None:
        pdf.savefig(fig, facecolor=BG_COLOR)
        plt.close(fig)
    else:
        plt.show()

def _make_bar_table_axes(fig, subplot_spec, width_ratios=(1.10, 1.00), wspace=0.05):
    inner = GridSpecFromSubplotSpec(
        1, 2, subplot_spec=subplot_spec,
        width_ratios=list(width_ratios),
        wspace=wspace
    )
    ax_bar = fig.add_subplot(inner[0, 0])
    ax_tbl = fig.add_subplot(inner[0, 1])
    return ax_bar, ax_tbl

def _draw_hbar_table_panel(
    ax_bar,
    ax_tbl,
    title,
    row_labels,
    bar_vals,
    band,
    xlabel,
    col_headers,
    col_values,
    pct_col_index=0,
    show_bar_value_labels=True,
):
    style_axis(ax_bar)
    ax_bar.set_title(title, fontsize=13, pad=6)

    row_labels = [str(x) for x in row_labels]
    vals = np.asarray(bar_vals, dtype=float)
    n = len(row_labels)

    row_fs = 11 if n <= 18 else 10 if n <= 24 else 9 if n <= 30 else 8
    hdr_fs = 10 if n <= 24 else 9

    if n == 0:
        ax_bar.axis("off")
        ax_tbl.axis("off")
        return

    y = np.arange(n)

    ax_bar.axvline(0.0, color=MUTED, linewidth=1, linestyle="--")
    ax_bar.axvspan(-band, band, color=ACCENT, alpha=0.10, zorder=0)
    ax_bar.axvline(-band, color=MUTED, linewidth=1, linestyle=":")
    ax_bar.axvline( band, color=MUTED, linewidth=1, linestyle=":")

    missing_mask = ~np.isfinite(vals)
    plot_vals = vals.copy()
    plot_vals[missing_mask] = 0.0

    colors_rgba = []
    for v, miss in zip(vals, missing_mask):
        if miss:
            colors_rgba.append(mcolors.to_rgba(MUTED, 0.35))
        elif abs(v) > band:
            colors_rgba.append(mcolors.to_rgba(RISK, 0.90))
        else:
            colors_rgba.append(mcolors.to_rgba(ACCENT, 0.90))

    ax_bar.barh(y, plot_vals, color=colors_rgba)

    ax_bar.set_yticks(y)
    ax_bar.set_yticklabels([""] * n)
    ax_bar.invert_yaxis()
    ax_bar.set_xlabel(xlabel, fontsize=11)

    finite = plot_vals[np.isfinite(plot_vals)]
    lim = np.nanmax(np.abs(finite)) if finite.size else band
    lim = max(band * 1.5, lim * 1.35, 15.0)
    ax_bar.set_xlim(-lim, lim)
    ax_bar.grid(axis="x", color=GRID_COLOR, alpha=0.5)

    if show_bar_value_labels:
        pad = lim * 0.03
        for yi, v, miss in zip(y, vals, missing_mask):
            if miss:
                continue
            if v >= 0:
                x = v + pad; ha = "left"
                if x > lim - pad:
                    x = v - pad; ha = "right"
            else:
                x = v - pad; ha = "right"
                if x < -lim + pad:
                    x = v + pad; ha = "left"
            ax_bar.text(x, yi, f"{v:+.1f}%", va="center", ha=ha, color=FG_COLOR, fontsize=9, clip_on=True)

    ax_tbl.set_facecolor(BG_COLOR)
    ax_tbl.set_xlim(0.0, 1.0)
    ax_tbl.set_ylim(-0.5, n - 0.5)
    ax_tbl.invert_yaxis()
    ax_tbl.axis("off")

    ax_tbl.text(0.00, 1.02, col_headers[0], color=MUTED, fontsize=hdr_fs, transform=ax_tbl.transAxes, ha="left")
    if len(col_headers) >= 2:
        ax_tbl.text(0.45, 1.02, col_headers[1], color=MUTED, fontsize=hdr_fs, transform=ax_tbl.transAxes, ha="right")
    if len(col_headers) >= 3:
        ax_tbl.text(0.70, 1.02, col_headers[2], color=MUTED, fontsize=hdr_fs, transform=ax_tbl.transAxes, ha="right")
    if len(col_headers) >= 4:
        ax_tbl.text(0.93, 1.02, col_headers[3], color=MUTED, fontsize=hdr_fs, transform=ax_tbl.transAxes, ha="right")

    for i, (lab, vals_tuple) in enumerate(zip(row_labels, col_values)):
        pct_like = np.nan
        if 0 <= pct_col_index < len(vals_tuple):
            pct_like = vals_tuple[pct_col_index]

        if not np.isfinite(pct_like):
            pct_color = MUTED
        elif abs(pct_like) <= band:
            pct_color = ACCENT
        else:
            pct_color = RISK

        ax_tbl.text(0.00, i, lab, color=FG_COLOR, fontsize=row_fs, transform=ax_tbl.transData, ha="left", va="center")

        if len(vals_tuple) >= 1:
            v = vals_tuple[0]
            ax_tbl.text(
                0.45, i,
                f"{v:+.1f}%" if np.isfinite(v) else "-",
                color=pct_color, fontsize=row_fs, transform=ax_tbl.transData, ha="right", va="center"
            )
        if len(vals_tuple) >= 2:
            v = vals_tuple[1]
            ax_tbl.text(
                0.70, i,
                fmt_num(v),
                color=FG_COLOR, fontsize=row_fs, transform=ax_tbl.transData, ha="right", va="center"
            )
        if len(vals_tuple) >= 3:
            v = vals_tuple[2]
            ax_tbl.text(
                0.93, i,
                fmt_num(v),
                color=FG_COLOR, fontsize=row_fs, transform=ax_tbl.transData, ha="right", va="center"
            )

def generate_metric_visualizations(
    csv_path=COMPLETE_CSV,
    out_dir="reports",
    pdf_name="metric_visualizations.pdf",
    make_pdf=True,
    PCT_BAND=OUTLIER_THRESHOLD,
    ASYM_BAND=OUTLIER_THRESHOLD,
    TREAT_ZERO_AS_MISSING=True,
    MAX_PANELS_PER_GROUP=16,
    GRID_COLS=4,
    ADD_PER_METRIC_PAGES=True,
    generated_ts=None,
):
    df = pd.read_csv(csv_path)
    long = build_long_outlier_table(df, pct_band=PCT_BAND, asym_band=ASYM_BAND, treat_zero_as_missing=TREAT_ZERO_AS_MISSING)

    if generated_ts is None:
        generated_ts = _make_generated_ts()


    def diag_limits(x, y):
        arr = np.r_[x, y]
        arr = arr[np.isfinite(arr)]
        if arr.size == 0:
            return (0, 1)
        mn, mx = float(np.min(arr)), float(np.max(arr))
        if mn == mx:
            mn -= 1
            mx += 1
        pad = 0.05 * (mx - mn)
        return (mn - pad, mx + pad)

    def draw_pct_band(ax, lo, hi, pct_band):
        band = pct_band / 100.0
        x_line = np.linspace(lo, hi, 200)
        y_lo = (1.0 - band) * x_line
        y_hi = (1.0 + band) * x_line
        ax.fill_between(x_line, y_lo, y_hi, color=ACCENT, alpha=0.08, zorder=0)
        ax.plot(x_line, y_lo, linewidth=1, linestyle=":", color=mcolors.to_rgba(MUTED, 0.9))
        ax.plot(x_line, y_hi, linewidth=1, linestyle=":", color=mcolors.to_rgba(MUTED, 0.9))

    def band_for(group: str) -> float:
        return band_for_group(group, PCT_BAND, ASYM_BAND)

    def plot_group_small_multiples(long_df, group, pdf=None):
        subg = long_df[long_df["group"] == group].copy()
        if subg.empty:
            return

        out_counts = subg.groupby("metric")["is_outlier"].sum().sort_values(ascending=False)
        metric_order = out_counts[out_counts > 0].index.tolist()

        remaining = [m for m in subg["metric"].unique().tolist() if m not in metric_order]
        if remaining:
            var_order = (subg.groupby("metric")["pct"].var().sort_values(ascending=False).index.tolist())
            metric_order += [m for m in var_order if m in remaining]

        metric_order = metric_order[:MAX_PANELS_PER_GROUP]
        n = len(metric_order)
        cols = GRID_COLS
        rows = int(np.ceil(n / cols))

        fig, axes = plt.subplots(rows, cols, figsize=(cols * 5.2, rows * 5.0), facecolor=BG_COLOR)
        axes = np.atleast_1d(axes).ravel()

        is_asym = group.startswith("NORDBORD ASYMMETRY")
        band = band_for(group)

        for i, m in enumerate(metric_order):
            ax = axes[i]
            style_axis(ax)

            sm = subg[subg["metric"] == m].copy()
            x = sm["x"].to_numpy(dtype=float)
            y = sm["y"].to_numpy(dtype=float)

            ok = np.isfinite(x) & np.isfinite(y)
            x_ok, y_ok = x[ok], y[ok]
            o = sm.loc[ok, "is_outlier"].to_numpy(dtype=bool)

            if x_ok.size:
                lo, hi = diag_limits(x_ok, y_ok)
                draw_pct_band(ax, lo, hi, pct_band=band)
                ax.plot([lo, hi], [lo, hi], linewidth=1.2, color=mcolors.to_rgba(MUTED, 0.9))

            ax.scatter(x_ok[~o], y_ok[~o], s=14, color=mcolors.to_rgba(MUTED, 0.35), edgecolors="none")
            ax.scatter(x_ok[o],  y_ok[o],  s=50, marker="x", linewidths=2.4, color=mcolors.to_rgba(RISK, 0.95))

            out_n = int(sm["is_outlier"].sum())
            ax.set_title(f"{m}" + (f"  [{out_n} Outlier(s)]" if out_n else ""), fontsize=11)

            if is_asym:
                ax.set_xlabel("Right", fontsize=10)
                ax.set_ylabel("Left", fontsize=10)
            else:
                ax.set_xlabel("Average", fontsize=10)
                ax.set_ylabel("Recent", fontsize=10)

            ax.grid(color=GRID_COLOR, alpha=0.35)

        for j in range(n, len(axes)):
            axes[j].axis("off")

        if is_asym:
            note = f"X = outliers, outlier if |asym| > {band:.0f}% (shaded band ≈ ±{band:.0f}%)"
        else:
            note = f"X = outliers, outlier if |% change| > {band:.0f}% (shaded band = ±{band:.0f}%)"

        fig.suptitle(f"{group}\n{note}", y=0.995, fontsize=16, color=FG_COLOR)
        fig.tight_layout(rect=[0, 0, 1, 0.94])

        if pdf is not None:
            pdf.savefig(fig, facecolor=BG_COLOR)
            plt.close(fig)
        else:
            plt.show()

    def plot_per_metric_pages(long_df, pdf=None):
        if long_df.empty:
            return

        def valid(x):
            if pd.isna(x) or not np.isfinite(x):
                return False
            if TREAT_ZERO_AS_MISSING and float(x) == 0.0:
                return False
            return True

        def safe_ratio_team(recent, baseline):
            if (not valid(recent)) or (not valid(baseline)) or abs(float(baseline)) < 1e-9:
                return np.nan
            r = float(recent) / float(baseline)
            return r if np.isfinite(r) else np.nan

        def pct_from_ratio_team(r):
            return np.nan if pd.isna(r) else (float(r) - 1.0) * 100.0

        def calc_asym_team(L, R):
            if pd.isna(L) or pd.isna(R) or abs(float(L) + float(R)) < 1e-9:
                return np.nan
            return (float(R) - float(L)) / ((float(R) + float(L)) / 2.0) * 100.0

        cat_label_to_base = {label: base for base, label in CATAPULT_METRICS}
        fd_label_to_base  = {label: base for base, label in FORCEDECKS_METRICS}
        nb_label_to_suf   = {label: suf  for suf,  label in NORDBORD_PAIRS}

        group_order = [
            "CATAPULT",
            "FORCEDECKS",
            "NORDBORD",
            "NORDBORD ASYMMETRY (RECENT)",
            "NORDBORD ASYMMETRY (AVG)",
        ]

        for group in group_order:
            subg = long_df[long_df["group"] == group].copy()
            if subg.empty:
                continue

            band = band_for(group)
            out_counts = subg.groupby("metric")["is_outlier"].sum().sort_values(ascending=False)
            metric_order = out_counts[out_counts > 0].index.tolist()

            remaining = [m for m in subg["metric"].unique().tolist() if m not in metric_order]
            metric_order += sorted(remaining)

            is_asym = group.startswith("NORDBORD ASYMMETRY")
            xlabel = "<- LEFT HEAVY | RIGHT HEAVY ->" if is_asym else "<- BELOW AVERAGE | ABOVE AVERAGE ->"

            for metric in metric_order:
                rows = []

                if group == "CATAPULT":
                    base = cat_label_to_base.get(metric)
                    if not base:
                        continue
                    a_col = f"{base}_avg"
                    r_col = f"{base}_recent"
                    if a_col not in df.columns or r_col not in df.columns:
                        continue

                    for _, r in df.iterrows():
                        player = str(r.get("player_name", "Unknown"))
                        a = r.get(a_col, np.nan)
                        rec = r.get(r_col, np.nan)
                        pct = pct_from_ratio_team(safe_ratio_team(rec, a))

                        if not valid(a):   a = np.nan
                        if not valid(rec): rec = np.nan

                        rows.append((player, pct, a, rec))

                    col_headers = ["Player", "% change", "Avg", "Recent"]

                elif group == "FORCEDECKS":
                    base = fd_label_to_base.get(metric)
                    if not base:
                        continue
                    a_col = f"{base}_avg"
                    r_col = f"{base}_recent"
                    if a_col not in df.columns or r_col not in df.columns:
                        continue

                    for _, r in df.iterrows():
                        player = str(r.get("player_name", "Unknown"))
                        a = r.get(a_col, np.nan)
                        rec = r.get(r_col, np.nan)
                        pct = pct_from_ratio_team(safe_ratio_team(rec, a))

                        if not valid(a):   a = np.nan
                        if not valid(rec): rec = np.nan

                        rows.append((player, pct, a, rec))

                    col_headers = ["Player", "% change", "Avg", "Recent"]

                elif group == "NORDBORD":
                    side = None
                    base_label = metric
                    if metric.startswith("Left "):
                        side = "left"
                        base_label = metric[len("Left "):]
                    elif metric.startswith("Right "):
                        side = "right"
                        base_label = metric[len("Right "):]

                    suf = nb_label_to_suf.get(base_label)
                    if (side is None) or (suf is None):
                        continue

                    a_col = f"{side}{suf}_avg"
                    r_col = f"{side}{suf}_recent"
                    if a_col not in df.columns or r_col not in df.columns:
                        continue

                    for _, r in df.iterrows():
                        player = str(r.get("player_name", "Unknown"))
                        a = r.get(a_col, np.nan)
                        rec = r.get(r_col, np.nan)
                        pct = pct_from_ratio_team(safe_ratio_team(rec, a))

                        if not valid(a):   a = np.nan
                        if not valid(rec): rec = np.nan

                        rows.append((player, pct, a, rec))

                    col_headers = ["Player", "% change", "Avg", "Recent"]

                elif group in ("NORDBORD ASYMMETRY (RECENT)", "NORDBORD ASYMMETRY (AVG)"):
                    suf = nb_label_to_suf.get(metric)
                    if suf is None:
                        continue

                    if group.endswith("(RECENT)"):
                        l_col = f"left{suf}_recent"
                        r_col = f"right{suf}_recent"
                    else:
                        l_col = f"left{suf}_avg"
                        r_col = f"right{suf}_avg"

                    if l_col not in df.columns or r_col not in df.columns:
                        continue

                    for _, r in df.iterrows():
                        player = str(r.get("player_name", "Unknown"))
                        L = r.get(l_col, np.nan)
                        R = r.get(r_col, np.nan)
                        a = calc_asym_team(L, R)

                        if not valid(L): L = np.nan
                        if not valid(R): R = np.nan

                        rows.append((player, a, L, R))

                    col_headers = ["Player", "Asym", "Left", "Right"]

                else:
                    continue

                if not rows:
                    continue

                tmp = pd.DataFrame(rows, columns=["player", "pct", "c2", "c3"])
                tmp.sort_values("pct", inplace=True, na_position="last")

                players_only = tmp["player"].tolist()
                pcts_only    = tmp["pct"].to_numpy(dtype=float)
                c2_only      = tmp["c2"].to_numpy(dtype=float)
                c3_only      = tmp["c3"].to_numpy(dtype=float)

                # Average row (computed over players with finite values)
                avg_pct = np.nanmean(pcts_only) if np.any(np.isfinite(pcts_only)) else np.nan
                avg_c2  = np.nanmean(c2_only)  if np.any(np.isfinite(c2_only))  else np.nan
                avg_c3  = np.nanmean(c3_only)  if np.any(np.isfinite(c3_only))  else np.nan

                players = players_only + ["AVERAGE"]
                pcts    = np.concatenate([pcts_only, [avg_pct]])
                col_vals = list(zip(pcts_only, c2_only, c3_only)) + [(avg_pct, avg_c2, avg_c3)]

                fig = plt.figure(figsize=(18, 9), facecolor=BG_COLOR)
                gs2 = GridSpec(1, 1, figure=fig)
                ax_bar, ax_tbl = _make_bar_table_axes(fig, gs2[0, 0], width_ratios=(1.20, 1.00), wspace=0.05)

                _draw_hbar_table_panel(
                    ax_bar, ax_tbl,
                    title="",
                    row_labels=players,
                    bar_vals=pcts,
                    band=band,
                    xlabel=xlabel,
                    col_headers=col_headers,
                    col_values=col_vals,
                    pct_col_index=0,
                    show_bar_value_labels=False,
                )

                ax_bar.grid(axis="x", color=GRID_COLOR, alpha=0.45)
                ax_bar.margins(y=0.02)

                out_n = int(np.nansum(np.abs(pcts_only) > band))
                n_tot = int(np.sum(np.isfinite(pcts_only)))

                fig.suptitle(
                    f"{group} - {metric}    |    Band: ±{band:.0f}%    |    Outliers: {out_n}/{n_tot if n_tot else 0}",
                    y=0.98,
                    fontsize=16,
                    color=FG_COLOR
                )

                fig.tight_layout(rect=[0, 0, 1, 0.94])
                if pdf is not None:
                    pdf.savefig(fig, facecolor=BG_COLOR)
                    plt.close(fig)
                else:
                    plt.show()

    os.makedirs(out_dir, exist_ok=True)
    pdf_path = os.path.join(out_dir, pdf_name)

    if make_pdf:
        with PdfPages(pdf_path) as pdf:
            _add_text_page(pdf, "METRIC VISUALIZATIONS", subtitle=generated_ts)
            _add_how_to_page(pdf, [
                "Use metric rankings to identify metrics with the most outliers",
                "Then pick a metric you'd like to look into and locate it under metric dashboards",
            ])

            _add_text_page(pdf, "METRIC RANKINGS", "Identify metrics with most outliers")

            plot_group_small_multiples(long, "CATAPULT",                    pdf=pdf)
            plot_group_small_multiples(long, "FORCEDECKS",                  pdf=pdf)
            plot_group_small_multiples(long, "NORDBORD",                    pdf=pdf)
            plot_group_small_multiples(long, "NORDBORD ASYMMETRY (RECENT)",  pdf=pdf)
            plot_group_small_multiples(long, "NORDBORD ASYMMETRY (AVG)",     pdf=pdf)

            if ADD_PER_METRIC_PAGES:
                _add_text_page(pdf, "METRIC DASHBOARDS", "Identify outliers for each metric")
                plot_per_metric_pages(long, pdf=pdf)
    else:
        plot_metric_rankings_page(long, pdf=None, top_n=12, pct_band=PCT_BAND, asym_band=ASYM_BAND)
        plot_group_small_multiples(long, "CATAPULT",                    pdf=None)
        plot_group_small_multiples(long, "FORCEDECKS",                  pdf=None)
        plot_group_small_multiples(long, "NORDBORD",                    pdf=None)
        plot_group_small_multiples(long, "NORDBORD ASYMMETRY (RECENT)",  pdf=None)
        plot_group_small_multiples(long, "NORDBORD ASYMMETRY (AVG)",     pdf=None)
        if ADD_PER_METRIC_PAGES:
            plot_per_metric_pages(long, pdf=None)

def sanitize_filename(name):
    name = re.sub(r"[^\w\-_\. ]", "", str(name)).strip().replace(" ", "_")
    return name if name else "player"

def _bar_table_panel(fig, gs_slot, title, labels, pct_vals, avg_vals, rec_vals,
                     band, x_label,
                     tbl_headers=("Metric", "% change", "Avg", "Recent")):

    sub = gs_slot.subgridspec(1, 2, width_ratios=[1.05, 1.00], wspace=0.06)
    ax_bar = fig.add_subplot(sub[0, 0])
    ax_tbl = fig.add_subplot(sub[0, 1])

    style_axis(ax_bar)
    n = len(labels)
    y = np.arange(n)

    pct = np.asarray(pct_vals, dtype=float)
    missing = ~np.isfinite(pct)
    plot_pct = pct.copy()
    plot_pct[missing] = 0.0

    ax_bar.axvline(0.0, color=MUTED, linewidth=1, linestyle="--")
    ax_bar.axvspan(-band, band, color=ACCENT, alpha=0.10, zorder=0)
    ax_bar.axvline(-band, color=MUTED, linewidth=1, linestyle=":")
    ax_bar.axvline(+band, color=MUTED, linewidth=1, linestyle=":")

    colors_rgba = []
    for v, miss in zip(pct, missing):
        if miss:
            colors_rgba.append(mcolors.to_rgba(MUTED, 0.35))
        elif abs(v) > band:
            colors_rgba.append(mcolors.to_rgba(RISK, 0.90))
        else:
            colors_rgba.append(mcolors.to_rgba(ACCENT, 0.90))

    ax_bar.barh(y, plot_pct, color=colors_rgba, height=0.62)

    ax_bar.set_yticks(y)
    ax_bar.set_yticklabels([""] * n)
    ax_bar.invert_yaxis()

    finite = plot_pct[np.isfinite(plot_pct)]
    lim = np.nanmax(np.abs(finite)) if finite.size else band
    lim = max(band * 1.5, lim * 1.35, 15.0)
    ax_bar.set_xlim(-lim, lim)

    pad = lim * 0.03
    for yi, v, miss in zip(y, pct, missing):
        if miss:
            ax_bar.text(pad, yi, "-", va="center", ha="left", color=MUTED, fontsize=10, clip_on=True)
            continue
        if v >= 0:
            x = v + pad; ha = "left"
            if x > lim - pad:
                x = v - pad; ha = "right"
        else:
            x = v - pad; ha = "right"
            if x < -lim + pad:
                x = v + pad; ha = "left"
        ax_bar.text(x, yi, f"{v:+.1f}%", va="center", ha=ha, color=FG_COLOR, fontsize=9, clip_on=True)

    ax_bar.grid(axis="x", color=GRID_COLOR, alpha=0.5)
    ax_bar.set_xlabel(x_label, fontsize=10, labelpad=8)
    ax_bar.set_title(title, fontsize=13, pad=6)

    ax_tbl.set_facecolor(BG_COLOR)
    ax_tbl.axis("off")
    ax_tbl.set_xlim(0.0, 1.0)
    ax_tbl.set_ylim(-1.2, n - 0.5)
    ax_tbl.invert_yaxis()

    h_metric, h_pct, h_avg, h_rec = tbl_headers
    ax_tbl.text(0.00, -0.95, h_metric, color=MUTED, fontsize=10, va="center")
    ax_tbl.text(0.53, -0.95, h_pct,    color=MUTED, fontsize=10, va="center", ha="right")
    ax_tbl.text(0.78, -0.95, h_avg,    color=MUTED, fontsize=10, va="center", ha="right")
    ax_tbl.text(1.00, -0.95, h_rec,    color=MUTED, fontsize=10, va="center", ha="right")

    for yi, lab, pv, av, rv in zip(y, labels, pct, avg_vals, rec_vals):
        if not np.isfinite(pv):
            pct_color = MUTED
        elif abs(pv) <= band:
            pct_color = ACCENT
        else:
            pct_color = RISK

        ax_tbl.text(0.00, yi, str(lab), color=FG_COLOR, fontsize=11, va="center")
        ax_tbl.text(0.53, yi, f"{pv:+.1f}%" if np.isfinite(pv) else "-",
                    color=pct_color, fontsize=11, va="center", ha="right")
        ax_tbl.text(0.78, yi, fmt_num(av), color=FG_COLOR, fontsize=11, va="center", ha="right")
        ax_tbl.text(1.00, yi, fmt_num(rv), color=FG_COLOR, fontsize=11, va="center", ha="right")

    return ax_bar, ax_tbl

def generate_player_dashboard_pages(
    df,
    pdf,
    PCT_HIGHLIGHT=OUTLIER_THRESHOLD,
    ASYM_FLAG_PCT=OUTLIER_THRESHOLD,
):
    def has_metric(base):
        return (f"{base}_avg" in df.columns) and (f"{base}_recent" in df.columns)

    catapult_metrics = [(b, l) for b, l in CATAPULT_METRICS if has_metric(b)]
    forcedecks_metrics = [(b, l) for b, l in FORCEDECKS_METRICS if has_metric(b)]

    for _, row in df.iterrows():
        player = row.get("player_name", "Unknown")
        player_display = str(player).upper()

        fig = plt.figure(figsize=(16, 14), facecolor=BG_COLOR)

        gs = GridSpec(
            6, 1, figure=fig,
            height_ratios=[0.80, 2.60, 1.60, 2.55, 1.70, 1.70],
            hspace=0.80
        )

        fig.subplots_adjust(top=0.97, bottom=0.04, left=0.05, right=0.97)

        ax_title = fig.add_subplot(gs[0, 0])

        total_metrics = 0
        optimal_metrics = 0
        all_complete = True

        for base, _ in catapult_metrics:
            rec = row.get(f"{base}_recent", np.nan)
            avg = row.get(f"{base}_avg", np.nan)
            if not (is_valid_nonzero(rec) and is_valid_nonzero(avg)):
                all_complete = False; break
            pct = pct_from_ratio(safe_ratio(rec, avg))
            if not np.isfinite(pct):
                all_complete = False; break
            total_metrics += 1
            if abs(pct) <= PCT_HIGHLIGHT:
                optimal_metrics += 1

        if all_complete:
            for base, _ in forcedecks_metrics:
                rec = row.get(f"{base}_recent", np.nan)
                avg = row.get(f"{base}_avg", np.nan)
                if not (is_valid_nonzero(rec) and is_valid_nonzero(avg)):
                    all_complete = False; break
                pct = pct_from_ratio(safe_ratio(rec, avg))
                if not np.isfinite(pct):
                    all_complete = False; break
                total_metrics += 1
                if abs(pct) <= PCT_HIGHLIGHT:
                    optimal_metrics += 1

        if all_complete:
            for suf, _label in NORDBORD_PAIRS:
                for side in ("left", "right"):
                    a = row.get(f"{side}{suf}_avg", np.nan)
                    r = row.get(f"{side}{suf}_recent", np.nan)
                    if not (is_valid_nonzero(r) and is_valid_nonzero(a)):
                        all_complete = False; break
                    pct = pct_from_ratio(safe_ratio(r, a))
                    if not np.isfinite(pct):
                        all_complete = False; break
                    total_metrics += 1
                    if abs(pct) <= PCT_HIGHLIGHT:
                        optimal_metrics += 1
                if not all_complete:
                    break

        if all_complete:
            for suf, _label in NORDBORD_PAIRS:
                lr = row.get(f"left{suf}_recent", np.nan)
                rr = row.get(f"right{suf}_recent", np.nan)
                if not (is_valid_nonzero(lr) and is_valid_nonzero(rr)):
                    all_complete = False; break
                a = calc_asym(lr, rr)
                if not np.isfinite(a):
                    all_complete = False; break
                total_metrics += 1
                if abs(a) <= ASYM_FLAG_PCT:
                    optimal_metrics += 1

        if all_complete:
            for suf, _label in NORDBORD_PAIRS:
                la = row.get(f"left{suf}_avg", np.nan)
                ra = row.get(f"right{suf}_avg", np.nan)
                if not (is_valid_nonzero(la) and is_valid_nonzero(ra)):
                    all_complete = False; break
                a = calc_asym(la, ra)
                if not np.isfinite(a):
                    all_complete = False; break
                total_metrics += 1
                if abs(a) <= ASYM_FLAG_PCT:
                    optimal_metrics += 1

        ax_title.axis("off")
        ax_title.set_facecolor(BG_COLOR)

        Lname = len(player_display)
        name_fs = 40 if Lname <= 18 else 34 if Lname <= 24 else 30

        ax_title.text(
            0.00, 0.72, player_display,
            fontsize=name_fs, fontweight="bold", color=FG_COLOR,
            transform=ax_title.transAxes, va="center", ha="left"
        )
        if all_complete and total_metrics > 0:
            ax_title.text(
                0.00, 0.00, f"{optimal_metrics}/{total_metrics} OPTIMAL METRICS",
                fontsize=18, fontweight="bold", color=MUTED,
                transform=ax_title.transAxes, va="center", ha="left"
            )

        cat_labels, cat_pcts, cat_avgs, cat_recs = [], [], [], []
        for base, label in catapult_metrics:
            a = row.get(f"{base}_avg", np.nan)
            r = row.get(f"{base}_recent", np.nan)
            cat_labels.append(label)
            cat_avgs.append(a)
            cat_recs.append(r)
            cat_pcts.append(pct_from_ratio(safe_ratio(r, a)))

        if cat_labels:
            _bar_table_panel(
                fig, gs[1, 0],
                title="CATAPULT",
                labels=cat_labels,
                pct_vals=cat_pcts,
                avg_vals=cat_avgs,
                rec_vals=cat_recs,
                band=PCT_HIGHLIGHT,
                x_label="<- BELOW AVERAGE | ABOVE AVERAGE ->",
                tbl_headers=("Metric", "% change", "Avg", "Recent")
            )
        else:
            ax = fig.add_subplot(gs[1, 0]); ax.axis("off"); ax.set_facecolor(BG_COLOR)

        fd_labels, fd_pcts, fd_avgs, fd_recs = [], [], [], []
        for base, label in forcedecks_metrics:
            a = row.get(f"{base}_avg", np.nan)
            r = row.get(f"{base}_recent", np.nan)
            fd_labels.append(label)
            fd_avgs.append(a)
            fd_recs.append(r)
            fd_pcts.append(pct_from_ratio(safe_ratio(r, a)))

        if fd_labels:
            _bar_table_panel(
                fig, gs[2, 0],
                title="FORCEDECKS",
                labels=fd_labels,
                pct_vals=fd_pcts,
                avg_vals=fd_avgs,
                rec_vals=fd_recs,
                band=PCT_HIGHLIGHT,
                x_label="<- BELOW AVERAGE | ABOVE AVERAGE ->",
                tbl_headers=("Metric", "% change", "Avg", "Recent")
            )
        else:
            ax = fig.add_subplot(gs[2, 0]); ax.axis("off"); ax.set_facecolor(BG_COLOR)

        nb_labels, nb_pcts, nb_avgs, nb_recs = [], [], [], []
        for suf, label in NORDBORD_PAIRS:
            for side, side_name in (("left", "Left"), ("right", "Right")):
                a = row.get(f"{side}{suf}_avg", np.nan)
                r = row.get(f"{side}{suf}_recent", np.nan)
                if (f"{side}{suf}_avg" in df.columns) and (f"{side}{suf}_recent" in df.columns):
                    nb_labels.append(f"{side_name} {label}")
                    nb_avgs.append(a)
                    nb_recs.append(r)
                    nb_pcts.append(pct_from_ratio(safe_ratio(r, a)))

        if nb_labels:
            _bar_table_panel(
                fig, gs[3, 0],
                title="NORDBORD",
                labels=nb_labels,
                pct_vals=nb_pcts,
                avg_vals=nb_avgs,
                rec_vals=nb_recs,
                band=PCT_HIGHLIGHT,
                x_label="<- BELOW AVERAGE | ABOVE AVERAGE ->",
                tbl_headers=("Metric", "% change", "Avg", "Recent")
            )
        else:
            ax = fig.add_subplot(gs[3, 0]); ax.axis("off"); ax.set_facecolor(BG_COLOR)

        ar_labels, ar_vals, ar_left, ar_right = [], [], [], []
        for suf, label in NORDBORD_PAIRS:
            lcol = f"left{suf}_recent"
            rcol = f"right{suf}_recent"
            if (lcol in df.columns) and (rcol in df.columns):
                L = row.get(lcol, np.nan)
                R = row.get(rcol, np.nan)
                ar_labels.append(label)
                ar_left.append(L)
                ar_right.append(R)
                ar_vals.append(calc_asym(L, R))

        if ar_labels:
            _bar_table_panel(
                fig, gs[4, 0],
                title="NORDBORD ASYMMETRY (RECENT)",
                labels=ar_labels,
                pct_vals=ar_vals,
                avg_vals=ar_left,
                rec_vals=ar_right,
                band=ASYM_FLAG_PCT,
                x_label="<- LEFT HEAVY | RIGHT HEAVY ->",
                tbl_headers=("Metric", "Asym", "Left", "Right")
            )
        else:
            ax = fig.add_subplot(gs[4, 0]); ax.axis("off"); ax.set_facecolor(BG_COLOR)

        aa_labels, aa_vals, aa_left, aa_right = [], [], [], []
        for suf, label in NORDBORD_PAIRS:
            lcol = f"left{suf}_avg"
            rcol = f"right{suf}_avg"
            if (lcol in df.columns) and (rcol in df.columns):
                L = row.get(lcol, np.nan)
                R = row.get(rcol, np.nan)
                aa_labels.append(label)
                aa_left.append(L)
                aa_right.append(R)
                aa_vals.append(calc_asym(L, R))

        if aa_labels:
            _bar_table_panel(
                fig, gs[5, 0],
                title="NORDBORD ASYMMETRY (AVERAGE)",
                labels=aa_labels,
                pct_vals=aa_vals,
                avg_vals=aa_left,
                rec_vals=aa_right,
                band=ASYM_FLAG_PCT,
                x_label="<- LEFT HEAVY | RIGHT HEAVY ->",
                tbl_headers=("Metric", "Asym", "Left", "Right")
            )
        else:
            ax = fig.add_subplot(gs[5, 0]); ax.axis("off"); ax.set_facecolor(BG_COLOR)

        if pdf is not None:
            pdf.savefig(fig, facecolor=BG_COLOR)
        plt.close(fig)

def generate_player_visualizations(
    csv_path=COMPLETE_CSV,
    out_dir="reports",
    pdf_name="player_visualizations.pdf",
    PCT_HIGHLIGHT=OUTLIER_THRESHOLD,
    ASYM_FLAG_PCT=OUTLIER_THRESHOLD,
    TREAT_ZERO_AS_MISSING=True,
    generated_ts=None,
):
    df = pd.read_csv(csv_path)
    long = build_long_outlier_table(
        df,
        pct_band=PCT_HIGHLIGHT,
        asym_band=ASYM_FLAG_PCT,
        treat_zero_as_missing=TREAT_ZERO_AS_MISSING
    )

    if generated_ts is None:
        generated_ts = _make_generated_ts()


    os.makedirs(out_dir, exist_ok=True)
    pdf_path = os.path.join(out_dir, pdf_name)

    with PdfPages(pdf_path) as pdf:
        _add_text_page(pdf, "PLAYER VISUALIZATIONS", subtitle=generated_ts)
        _add_how_to_page(pdf, [
            "Use player rankings to identify players with the most outliers",
            "Then pick a player you'd like to look into and locate it under player dashboards",
        ])

        _add_text_page(pdf, "PLAYER RANKINGS", "Identify players with most outliers")
        plot_player_rankings_page(long, pdf=pdf, top_n=10, pct_band=PCT_HIGHLIGHT, asym_band=ASYM_FLAG_PCT)

        _add_text_page(pdf, "PLAYER DASHBOARDS", "Identify outliers for each player")
        generate_player_dashboard_pages(df, pdf=pdf, PCT_HIGHLIGHT=PCT_HIGHLIGHT, ASYM_FLAG_PCT=ASYM_FLAG_PCT)

def _xl_argb(hex_color: str) -> str:
    if hex_color is None:
        return "FF000000"
    h = str(hex_color).strip().lstrip("#")
    if len(h) == 8:
        return h.upper()
    if len(h) != 6:
        return "FF000000"
    return ("FF" + h).upper()

def _safe_sheet_name(name: str, existing: set) -> str:
    s = re.sub(r'[:\\/?*\[\]]', ' ', str(name)).strip()
    s = re.sub(r'\s+', ' ', s)
    if len(s) == 0:
        s = "Sheet"
    s = s[:31]
    base = s
    k = 2
    while s in existing:
        suffix = f" {k}"
        s = (base[:31-len(suffix)] + suffix) if len(base) + len(suffix) > 31 else (base + suffix)
        k += 1
    existing.add(s)
    return s

def _apply_col_widths(ws, min_w=10, max_w=40, extra=2):
    for col in range(1, ws.max_column + 1):
        letter = get_column_letter(col)
        max_len = 0
        for cell in ws[letter]:
            v = cell.value
            if v is None:
                continue
            max_len = max(max_len, len(str(v)))
        ws.column_dimensions[letter].width = max(min_w, min(max_w, max_len + extra))

def _style_title_row(ws, row, text, n_cols, bg_hex, fg_hex, font_size=18):
    fill = PatternFill("solid", fgColor=_xl_argb(bg_hex))
    font = Font(bold=True, size=font_size, color=_xl_argb(fg_hex))
    align = Alignment(horizontal="center", vertical="center")
    for c in range(1, n_cols + 1):
        cell = ws.cell(row=row, column=c)
        cell.fill = fill
        cell.font = font
        cell.alignment = align
    if n_cols >= 2:
        ws.merge_cells(start_row=row, start_column=1, end_row=row, end_column=n_cols)
    ws.cell(row=row, column=1).value = text
    ws.row_dimensions[row].height = 28

def _style_subtitle_row(ws, row, text, n_cols, bg_hex, muted_hex, font_size=11):
    fill = PatternFill("solid", fgColor=_xl_argb(bg_hex))
    font = Font(size=font_size, color=_xl_argb(muted_hex))
    align = Alignment(horizontal="center", vertical="center")
    for c in range(1, n_cols + 1):
        cell = ws.cell(row=row, column=c)
        cell.fill = fill
        cell.font = font
        cell.alignment = align
    if n_cols >= 2:
        ws.merge_cells(start_row=row, start_column=1, end_row=row, end_column=n_cols)
    ws.cell(row=row, column=1).value = text
    ws.row_dimensions[row].height = 18

def _style_header_row(ws, row, headers, bg_hex, fg_hex, grid_hex):
    fill = PatternFill("solid", fgColor=_xl_argb(bg_hex))
    font = Font(bold=True, color=_xl_argb(fg_hex))
    align = Alignment(horizontal="left", vertical="center")
    side = Side(style="thin", color=_xl_argb(grid_hex))
    border = Border(top=side, bottom=side)
    for j, h in enumerate(headers, start=1):
        cell = ws.cell(row=row, column=j, value=h)
        cell.fill = fill
        cell.font = font
        cell.alignment = align
        cell.border = border
    ws.row_dimensions[row].height = 18

def _style_data_row(ws, row, values, n_cols, bg_hex, fg_hex, grid_hex,
                    pct_col=None, pct_val=None, band=None,
                    accent_hex=None, risk_hex=None, muted_hex=None):
    base_fill = PatternFill("solid", fgColor=_xl_argb(bg_hex))
    base_font = Font(color=_xl_argb(fg_hex))
    align_left = Alignment(horizontal="left", vertical="center")
    align_right = Alignment(horizontal="right", vertical="center")
    side = Side(style="thin", color=_xl_argb(grid_hex))
    border = Border(bottom=side)

    for j in range(1, n_cols + 1):
        cell = ws.cell(row=row, column=j, value=values[j-1] if j-1 < len(values) else "")
        cell.fill = base_fill
        cell.font = base_font
        cell.border = border
        cell.alignment = align_left if j == 1 else align_right

    if (pct_col is not None) and (1 <= pct_col <= n_cols):
        cell = ws.cell(row=row, column=pct_col)
        if isinstance(pct_val, (int, float)) and np.isfinite(pct_val):
            cell.number_format = '0.0"%"'
        if pct_val is None or (not (isinstance(pct_val, (int, float)) and np.isfinite(pct_val))):
            if muted_hex:
                cell.fill = PatternFill("solid", fgColor=_xl_argb(muted_hex))
                cell.font = Font(color=_xl_argb(BG_COLOR))
        else:
            if (band is not None) and (abs(float(pct_val)) <= float(band)):
                if accent_hex:
                    cell.fill = PatternFill("solid", fgColor=_xl_argb(accent_hex))
                    cell.font = Font(color=_xl_argb(BG_COLOR), bold=True)
            else:
                if risk_hex:
                    cell.fill = PatternFill("solid", fgColor=_xl_argb(risk_hex))
                    cell.font = Font(color=_xl_argb(BG_COLOR), bold=True)

def _metric_sheet_key(group: str, metric: str) -> str:
    short = {
        "CATAPULT": "CAT",
        "FORCEDECKS": "FD",
        "NORDBORD": "NB",
        "NORDBORD ASYMMETRY (RECENT)": "NB-ASYR",
        "NORDBORD ASYMMETRY (AVG)": "NB-ASYA",
    }.get(group, "MET")
    m = re.sub(r"\s+", " ", str(metric)).strip()
    m = m.replace(" | ", " ")
    return f"{short} {m}"

def _build_rank_tables(long_df, pct_band, asym_band):
    if long_df.empty:
        return (pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame())

    sub = long_df[np.isfinite(long_df["pct"].to_numpy(dtype=float))].copy()
    if sub.empty:
        return (pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame())

    sub["band"] = sub["group"].apply(lambda g: band_for_group(g, pct_band, asym_band)).astype(float)
    sub["is_out"] = (np.abs(sub["pct"].astype(float)) > sub["band"])
    sub["is_opt"] = (np.abs(sub["pct"].astype(float)) <= sub["band"])

    sub["metric_full"] = sub["group"].astype(str) + " | " + sub["metric"].astype(str)
    metric_base = sub.groupby("metric_full").agg(
        outlier_players=("is_out", "sum"),
        optimal_players=("is_opt", "sum"),
        players_with_data=("pct", "count"),
    ).reset_index()
    metric_base["group"] = metric_base["metric_full"].str.split(" \| ", n=1).str[0]
    metric_base["metric"] = metric_base["metric_full"].str.split(" \| ", n=1).str[1]
    metric_base = metric_base.drop(columns=["metric_full"])

    metric_out = metric_base.sort_values(["outlier_players", "players_with_data", "group", "metric"], ascending=[False, False, True, True]).reset_index(drop=True)
    metric_opt = metric_base.sort_values(["optimal_players", "players_with_data", "group", "metric"], ascending=[False, False, True, True]).reset_index(drop=True)

    player_base = sub.groupby("player").agg(
        outlier_metrics=("is_out", "sum"),
        optimal_metrics=("is_opt", "sum"),
        metrics_with_data=("pct", "count"),
    ).reset_index().rename(columns={"player": "player_name"})

    player_out = player_base.sort_values(["outlier_metrics", "metrics_with_data", "player_name"], ascending=[False, False, True]).reset_index(drop=True)
    player_opt = player_base.sort_values(["optimal_metrics", "metrics_with_data", "player_name"], ascending=[False, False, True]).reset_index(drop=True)

    return metric_out, metric_opt, player_out, player_opt

def generate_metric_spreadsheets(
    csv_path=COMPLETE_CSV,
    out_dir="reports",
    xlsx_name="metric_spreadsheets.xlsx",
    PCT_BAND=OUTLIER_THRESHOLD,
    ASYM_BAND=OUTLIER_THRESHOLD,
    TREAT_ZERO_AS_MISSING=True,
    generated_ts=None,
):

    df = pd.read_csv(csv_path)
    long = build_long_outlier_table(df, pct_band=PCT_BAND, asym_band=ASYM_BAND, treat_zero_as_missing=TREAT_ZERO_AS_MISSING)

    if generated_ts is None:
        generated_ts = _make_generated_ts()


    metric_out, metric_opt, _player_out, _player_opt = _build_rank_tables(long, PCT_BAND, ASYM_BAND)

    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, xlsx_name)

    wb = Workbook()
    ws0 = wb.active
    ws0.title = "Metric Rankings"

    n_cols = 7
    _style_title_row(ws0, 1, "METRIC SPREADSHEETS", n_cols, BG_COLOR, FG_COLOR, font_size=18)
    _style_subtitle_row(ws0, 2, generated_ts, n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 3, "Metric Rankings + Metric Dashboards (table view)", n_cols, BG_COLOR, MUTED, font_size=11)

    _style_title_row(ws0, 4, "HOW TO USE", n_cols, BG_COLOR, FG_COLOR, font_size=13)
    _style_subtitle_row(ws0, 5, "1) Use the ranking tables below to choose a metric to investigate.", n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 6, "2) Open that metric's tab to see the full player table for the metric.", n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 7, "3) Add context in Notes (injury, minutes restriction, position, etc.).", n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 8, "Color key: Green = within band | Red = outlier | Gray = missing/invalid", n_cols, BG_COLOR, MUTED, font_size=11)

    row = 10

    def write_rank_table(ws, start_row, title, df_rank, count_col, label_count):
        _style_title_row(ws, start_row, title, n_cols, BG_COLOR, FG_COLOR, font_size=14)
        headers = ["Rank", "Group", "Metric", label_count, "Players w/ data", "Rate", "Notes"]
        _style_header_row(ws, start_row + 1, headers, BG_COLOR, FG_COLOR, GRID_COLOR)

        r = start_row + 2
        for i, rr in df_rank.iterrows():
            players_w_data = int(rr.get("players_with_data", 0))
            count = int(rr.get(count_col, 0))
            rate = (count / players_w_data * 100.0) if players_w_data else np.nan

            vals = [
                int(i + 1),
                str(rr.get("group", "")),
                str(rr.get("metric", "")),
                count,
                players_w_data,
                (f"{rate:.1f}%" if np.isfinite(rate) else "-"),
                "",
            ]

            _style_data_row(ws, r, vals, n_cols, BG_COLOR, FG_COLOR, GRID_COLOR, pct_col=None)

            c_cell = ws.cell(row=r, column=4)
            if "OUTLIER" in title.upper():
                c_cell.fill = PatternFill("solid", fgColor=_xl_argb(RISK))
                c_cell.font = Font(color=_xl_argb(BG_COLOR), bold=True)
            else:
                c_cell.fill = PatternFill("solid", fgColor=_xl_argb(ACCENT))
                c_cell.font = Font(color=_xl_argb(BG_COLOR), bold=True)

            ws.cell(row=r, column=1).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=2).alignment = Alignment(horizontal="left",  vertical="center")
            ws.cell(row=r, column=3).alignment = Alignment(horizontal="left",  vertical="center")
            ws.cell(row=r, column=4).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=5).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=6).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=7).alignment = Alignment(horizontal="left",  vertical="center")

            r += 1

        return r + 2

    if not metric_out.empty:
        row = write_rank_table(ws0, row, "MOST OUTLIER PLAYERS (BY METRIC)", metric_out, "outlier_players", "Outlier players")
    else:
        _style_title_row(ws0, row, "MOST OUTLIER PLAYERS (BY METRIC)", n_cols, BG_COLOR, FG_COLOR, font_size=14)
        ws0.cell(row=row+1, column=1, value="(No data)").font = Font(color=_xl_argb(MUTED))
        row += 4

    if not metric_opt.empty:
        row = write_rank_table(ws0, row, "MOST OPTIMAL PLAYERS (BY METRIC)", metric_opt, "optimal_players", "Optimal players")
    else:
        _style_title_row(ws0, row, "MOST OPTIMAL PLAYERS (BY METRIC)", n_cols, BG_COLOR, FG_COLOR, font_size=14)
        ws0.cell(row=row+1, column=1, value="(No data)").font = Font(color=_xl_argb(MUTED))

    ws0.freeze_panes = "A9"
    _apply_col_widths(ws0, min_w=10, max_w=45, extra=2)

    existing = {ws0.title}

    def valid(x):
        if pd.isna(x) or not np.isfinite(x):
            return False
        if TREAT_ZERO_AS_MISSING and float(x) == 0.0:
            return False
        return True

    def safe_ratio_team(recent, baseline):
        if (not valid(recent)) or (not valid(baseline)) or abs(float(baseline)) < 1e-9:
            return np.nan
        r = float(recent) / float(baseline)
        return r if np.isfinite(r) else np.nan

    def pct_from_ratio_team(r):
        return np.nan if pd.isna(r) else (float(r) - 1.0) * 100.0

    def calc_asym_team(L, R):
        if pd.isna(L) or pd.isna(R) or abs(float(L) + float(R)) < 1e-9:
            return np.nan
        return (float(R) - float(L)) / ((float(R) + float(L)) / 2.0) * 100.0

    cat_label_to_base = {label: base for base, label in CATAPULT_METRICS}
    fd_label_to_base  = {label: base for base, label in FORCEDECKS_METRICS}
    nb_label_to_suf   = {label: suf  for suf,  label in NORDBORD_PAIRS}

    group_order = [
        "CATAPULT",
        "FORCEDECKS",
        "NORDBORD",
        "NORDBORD ASYMMETRY (RECENT)",
        "NORDBORD ASYMMETRY (AVG)",
    ]

    def group_band(g: str) -> float:
        return band_for_group(g, PCT_BAND, ASYM_BAND)

    order_counts = {}
    if not long.empty:
        tmp = long[np.isfinite(long["pct"].to_numpy(dtype=float))].copy()
        tmp["band"] = tmp["group"].apply(lambda g: group_band(g)).astype(float)
        tmp["is_out"] = (np.abs(tmp["pct"].astype(float)) > tmp["band"])
        for g in group_order:
            subg = tmp[tmp["group"] == g].copy()
            if subg.empty:
                continue
            out_counts = subg.groupby("metric", sort=True)["is_out"].sum().sort_values(ascending=False)
            order_counts[g] = out_counts

    for group in group_order:
        metric_list = []
        if group == "CATAPULT":
            metric_list = [label for _b, label in CATAPULT_METRICS if f"{_b}_avg" in df.columns and f"{_b}_recent" in df.columns]
        elif group == "FORCEDECKS":
            metric_list = [label for _b, label in FORCEDECKS_METRICS if f"{_b}_avg" in df.columns and f"{_b}_recent" in df.columns]
        elif group == "NORDBORD":
            metric_list = []
            for suf, label in NORDBORD_PAIRS:
                for side in ("Left", "Right"):
                    bcol = f"{side.lower()}{suf}_avg"
                    rcol = f"{side.lower()}{suf}_recent"
                    if bcol in df.columns and rcol in df.columns:
                        metric_list.append(f"{side} {label}")
        elif group.startswith("NORDBORD ASYMMETRY"):
            metric_list = [label for _suf, label in NORDBORD_PAIRS]
        else:
            metric_list = []

        if not metric_list:
            continue

        if group in order_counts:
            out_counts = order_counts[group]
            metric_order = [m for m in out_counts.index.tolist() if m in metric_list]
            remaining = [m for m in metric_list if m not in metric_order]
            metric_order += sorted(remaining)
        else:
            metric_order = sorted(metric_list)

        for metric in metric_order:
            band = group_band(group)
            is_asym = group.startswith("NORDBORD ASYMMETRY")
            headers = ["Player", ("% change" if not is_asym else "Asym"), ("Avg" if not is_asym else "Left"), ("Recent" if not is_asym else "Right"), "Notes"]

            rows = []
            if group == "CATAPULT":
                base = cat_label_to_base.get(metric)
                if not base:
                    continue
                a_col = f"{base}_avg"; r_col = f"{base}_recent"
                if a_col not in df.columns or r_col not in df.columns:
                    continue
                for _, r in df.iterrows():
                    player = str(r.get("player_name", "Unknown"))
                    a = r.get(a_col, np.nan); rec = r.get(r_col, np.nan)
                    pct = pct_from_ratio_team(safe_ratio_team(rec, a))
                    if not valid(a):   a = np.nan
                    if not valid(rec): rec = np.nan
                    rows.append((player, pct, a, rec))

            elif group == "FORCEDECKS":
                base = fd_label_to_base.get(metric)
                if not base:
                    continue
                a_col = f"{base}_avg"; r_col = f"{base}_recent"
                if a_col not in df.columns or r_col not in df.columns:
                    continue
                for _, r in df.iterrows():
                    player = str(r.get("player_name", "Unknown"))
                    a = r.get(a_col, np.nan); rec = r.get(r_col, np.nan)
                    pct = pct_from_ratio_team(safe_ratio_team(rec, a))
                    if not valid(a):   a = np.nan
                    if not valid(rec): rec = np.nan
                    rows.append((player, pct, a, rec))

            elif group == "NORDBORD":
                side = None
                base_label = metric
                if metric.startswith("Left "):
                    side = "left"; base_label = metric[len("Left "):]
                elif metric.startswith("Right "):
                    side = "right"; base_label = metric[len("Right "):]
                suf = nb_label_to_suf.get(base_label)
                if side is None or suf is None:
                    continue
                a_col = f"{side}{suf}_avg"; r_col = f"{side}{suf}_recent"
                if a_col not in df.columns or r_col not in df.columns:
                    continue
                for _, r in df.iterrows():
                    player = str(r.get("player_name", "Unknown"))
                    a = r.get(a_col, np.nan); rec = r.get(r_col, np.nan)
                    pct = pct_from_ratio_team(safe_ratio_team(rec, a))
                    if not valid(a):   a = np.nan
                    if not valid(rec): rec = np.nan
                    rows.append((player, pct, a, rec))

            elif group in ("NORDBORD ASYMMETRY (RECENT)", "NORDBORD ASYMMETRY (AVG)"):
                suf = nb_label_to_suf.get(metric)
                if suf is None:
                    continue
                if group.endswith("(RECENT)"):
                    l_col = f"left{suf}_recent"; r_col = f"right{suf}_recent"
                else:
                    l_col = f"left{suf}_avg"; r_col = f"right{suf}_avg"
                if l_col not in df.columns or r_col not in df.columns:
                    continue
                for _, r in df.iterrows():
                    player = str(r.get("player_name", "Unknown"))
                    L = r.get(l_col, np.nan); R = r.get(r_col, np.nan)
                    a = calc_asym_team(L, R)
                    if not valid(L): L = np.nan
                    if not valid(R): R = np.nan
                    rows.append((player, a, L, R))
            else:
                continue

            if not rows:
                continue

            tmp = pd.DataFrame(rows, columns=["Player", "Pct", "C2", "C3"])
            tmp.sort_values("Pct", inplace=True, na_position="last")

            sheet_title = _metric_sheet_key(group, metric)
            sheet_name = _safe_sheet_name(sheet_title, existing)
            ws = wb.create_sheet(title=sheet_name)

            _style_title_row(ws, 1, f"{group} - {metric}", 5, BG_COLOR, FG_COLOR, font_size=16)
            _style_subtitle_row(ws, 2, f"Band: ±{band:.0f}%   (green = within band, red = outlier, gray = missing)", 5, BG_COLOR, MUTED, font_size=11)
            _style_subtitle_row(
                ws, 3,
                "Excel: to sort players by another metric, right click the first row of the metric -> Sort -> Sort (Smallest to Largest // Largest to Smallest)",
                5, BG_COLOR, MUTED, font_size=10
            )
            _style_header_row(ws, 5, headers, BG_COLOR, FG_COLOR, GRID_COLOR)

            r0 = 6
            for i in range(len(tmp)):
                player = tmp.iloc[i]["Player"]
                pctv = tmp.iloc[i]["Pct"]
                c2 = tmp.iloc[i]["C2"]
                c3 = tmp.iloc[i]["C3"]

                def _disp(x):
                    return "-" if (x is None or pd.isna(x) or (isinstance(x, float) and not np.isfinite(x))) else float(x)

                vals = [
                    str(player),
                    (_disp(pctv) if _disp(pctv) == "-" else float(pctv)),
                    (_disp(c2) if _disp(c2) == "-" else float(c2)),
                    (_disp(c3) if _disp(c3) == "-" else float(c3)),
                    "",
                ]

                _style_data_row(
                    ws, r0 + i, vals, 5, BG_COLOR, FG_COLOR, GRID_COLOR,
                    pct_col=2, pct_val=(float(pctv) if (isinstance(pctv, (int, float)) and np.isfinite(pctv)) else None),
                    band=band, accent_hex=ACCENT, risk_hex=RISK, muted_hex=MUTED
                )

                ws.cell(row=r0 + i, column=5).alignment = Alignment(horizontal="left", vertical="center")

            avg_row = r0 + len(tmp)
            arr_pct = tmp["Pct"].to_numpy(dtype=float)
            arr_c2  = tmp["C2"].to_numpy(dtype=float)
            arr_c3  = tmp["C3"].to_numpy(dtype=float)
            avg_pct = np.nanmean(arr_pct) if np.any(np.isfinite(arr_pct)) else np.nan
            avg_c2  = np.nanmean(arr_c2)  if np.any(np.isfinite(arr_c2))  else np.nan
            avg_c3  = np.nanmean(arr_c3)  if np.any(np.isfinite(arr_c3))  else np.nan
            avg_vals = [
                "AVERAGE",
                ("-" if not np.isfinite(avg_pct) else float(avg_pct)),
                ("-" if not np.isfinite(avg_c2)  else float(avg_c2)),
                ("-" if not np.isfinite(avg_c3)  else float(avg_c3)),
                "Average across players with data",
            ]
            _style_data_row(
                ws, avg_row, avg_vals, 5, BG_COLOR, FG_COLOR, GRID_COLOR,
                pct_col=2, pct_val=(float(avg_pct) if np.isfinite(avg_pct) else None),
                band=band, accent_hex=ACCENT, risk_hex=RISK, muted_hex=MUTED
            )
            ws.cell(row=avg_row, column=5).alignment = Alignment(horizontal="left", vertical="center")
            for c in range(1, 5):
                color = BG_COLOR if c == 2 else FG_COLOR
                ws.cell(row=avg_row, column=c).font = Font(bold=True, color=_xl_argb(color))


            ws.freeze_panes = "A4"
            _apply_col_widths(ws, min_w=10, max_w=40, extra=2)

    wb.save(out_path)
    return out_path

def generate_player_spreadsheets(
    csv_path=COMPLETE_CSV,
    out_dir="reports",
    xlsx_name="player_spreadsheets.xlsx",
    PCT_BAND=OUTLIER_THRESHOLD,
    ASYM_BAND=OUTLIER_THRESHOLD,
    TREAT_ZERO_AS_MISSING=True,
    generated_ts=None,
):

    df = pd.read_csv(csv_path)
    long = build_long_outlier_table(df, pct_band=PCT_BAND, asym_band=ASYM_BAND, treat_zero_as_missing=TREAT_ZERO_AS_MISSING)

    if generated_ts is None:
        generated_ts = _make_generated_ts()


    _metric_out, _metric_opt, player_out, player_opt = _build_rank_tables(long, PCT_BAND, ASYM_BAND)

    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, xlsx_name)

    wb = Workbook()
    ws0 = wb.active
    ws0.title = "Player Rankings"

    n_cols = 6
    _style_title_row(ws0, 1, "PLAYER SPREADSHEETS", n_cols, BG_COLOR, FG_COLOR, font_size=18)
    _style_subtitle_row(ws0, 2, generated_ts, n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 3, "Player Rankings + Player Dashboards (table view)", n_cols, BG_COLOR, MUTED, font_size=11)

    _style_title_row(ws0, 4, "HOW TO USE", n_cols, BG_COLOR, FG_COLOR, font_size=13)
    _style_subtitle_row(ws0, 5, "1) Use the ranking tables below to choose a player to investigate.", n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 6, "2) Open that player's tab to see their full metric table.", n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 7, "3) Add context in Notes (injury, minutes restriction, position, etc.).", n_cols, BG_COLOR, MUTED, font_size=11)
    _style_subtitle_row(ws0, 8, "Color key: Green = within band | Red = outlier | Gray = missing/invalid", n_cols, BG_COLOR, MUTED, font_size=11)

    row = 10

    def write_rank_table(ws, start_row, title, df_rank, count_col, label_count):
        _style_title_row(ws, start_row, title, n_cols, BG_COLOR, FG_COLOR, font_size=14)
        headers = ["Rank", "Player", label_count, "Metrics w/ data", "Rate", "Notes"]
        _style_header_row(ws, start_row + 1, headers, BG_COLOR, FG_COLOR, GRID_COLOR)

        r = start_row + 2
        for i, rr in df_rank.iterrows():
            metrics_w_data = int(rr.get("metrics_with_data", 0))
            count = int(rr.get(count_col, 0))
            rate = (count / metrics_w_data * 100.0) if metrics_w_data else np.nan

            vals = [
                int(i + 1),
                str(rr.get("player_name", "")),
                count,
                metrics_w_data,
                (f"{rate:.1f}%" if np.isfinite(rate) else "-"),
                "",
            ]

            _style_data_row(ws, r, vals, n_cols, BG_COLOR, FG_COLOR, GRID_COLOR, pct_col=None)

            c_cell = ws.cell(row=r, column=3)
            if "OUTLIER" in title.upper():
                c_cell.fill = PatternFill("solid", fgColor=_xl_argb(RISK))
                c_cell.font = Font(color=_xl_argb(BG_COLOR), bold=True)
            else:
                c_cell.fill = PatternFill("solid", fgColor=_xl_argb(ACCENT))
                c_cell.font = Font(color=_xl_argb(BG_COLOR), bold=True)

            ws.cell(row=r, column=1).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=2).alignment = Alignment(horizontal="left",  vertical="center")
            ws.cell(row=r, column=3).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=4).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=5).alignment = Alignment(horizontal="right", vertical="center")
            ws.cell(row=r, column=6).alignment = Alignment(horizontal="left",  vertical="center")

            r += 1

        return r + 2

    if not player_out.empty:
        row = write_rank_table(ws0, row, "MOST OUTLIER METRICS (BY PLAYER)", player_out, "outlier_metrics", "Outlier metrics")
    else:
        _style_title_row(ws0, row, "MOST OUTLIER METRICS (BY PLAYER)", n_cols, BG_COLOR, FG_COLOR, font_size=14)
        ws0.cell(row=row+1, column=1, value="(No data)").font = Font(color=_xl_argb(MUTED))
        row += 4

    if not player_opt.empty:
        row = write_rank_table(ws0, row, "MOST OPTIMAL METRICS (BY PLAYER)", player_opt, "optimal_metrics", "Optimal metrics")
    else:
        _style_title_row(ws0, row, "MOST OPTIMAL METRICS (BY PLAYER)", n_cols, BG_COLOR, FG_COLOR, font_size=14)
        ws0.cell(row=row+1, column=1, value="(No data)").font = Font(color=_xl_argb(MUTED))

    ws0.freeze_panes = "A9"
    _apply_col_widths(ws0, min_w=10, max_w=45, extra=2)

    existing = {ws0.title}

    def valid(x):
        if pd.isna(x) or not np.isfinite(x):
            return False
        if TREAT_ZERO_AS_MISSING and float(x) == 0.0:
            return False
        return True

    def safe_ratio_team(recent, baseline):
        if (not valid(recent)) or (not valid(baseline)) or abs(float(baseline)) < 1e-9:
            return np.nan
        r = float(recent) / float(baseline)
        return r if np.isfinite(r) else np.nan

    def pct_from_ratio_team(r):
        return np.nan if pd.isna(r) else (float(r) - 1.0) * 100.0

    def calc_asym_team(L, R):
        if pd.isna(L) or pd.isna(R) or abs(float(L) + float(R)) < 1e-9:
            return np.nan
        return (float(R) - float(L)) / ((float(R) + float(L)) / 2.0) * 100.0

    def has_metric(base): return (f"{base}_avg" in df.columns) and (f"{base}_recent" in df.columns)

    catapult_metrics = [(b, l) for b, l in CATAPULT_METRICS if has_metric(b)]
    forcedecks_metrics = [(b, l) for b, l in FORCEDECKS_METRICS if has_metric(b)]

    def write_section(ws, start_row, title, headers, rows, band):
        _style_title_row(ws, start_row, title, len(headers), BG_COLOR, FG_COLOR, font_size=13)
        _style_header_row(ws, start_row + 1, headers, BG_COLOR, FG_COLOR, GRID_COLOR)

        r = start_row + 2
        for player_row in rows:
            label, pv, c2, c3 = player_row
            vals = [
                str(label),
                ("-" if (pv is None or pd.isna(pv) or (isinstance(pv, float) and not np.isfinite(pv))) else float(pv)),
                ("-" if (c2 is None or pd.isna(c2) or (isinstance(c2, float) and not np.isfinite(c2))) else float(c2)),
                ("-" if (c3 is None or pd.isna(c3) or (isinstance(c3, float) and not np.isfinite(c3))) else float(c3)),
                "",
            ]

            pv_num = float(pv) if (isinstance(pv, (int, float)) and np.isfinite(pv)) else None

            _style_data_row(
                ws, r, vals, len(headers), BG_COLOR, FG_COLOR, GRID_COLOR,
                pct_col=2, pct_val=pv_num, band=band,
                accent_hex=ACCENT, risk_hex=RISK, muted_hex=MUTED
            )

            ws.cell(row=r, column=len(headers)).alignment = Alignment(horizontal="left", vertical="center")

            r += 1

        return r + 1

    for _, rowp in df.iterrows():
        player = str(rowp.get("player_name", "Unknown"))
        sheet_name = _safe_sheet_name(player, existing)
        ws = wb.create_sheet(title=sheet_name)

        _style_title_row(ws, 1, player.upper(), 5, BG_COLOR, FG_COLOR, font_size=18)
        _style_subtitle_row(ws, 2, f"Band: ±{PCT_BAND:.0f}%  |  Asym band: ±{ASYM_BAND:.0f}%", 5, BG_COLOR, MUTED, font_size=11)

        cur = 4

        cat_rows = []
        for base, label in catapult_metrics:
            a = rowp.get(f"{base}_avg", np.nan)
            r = rowp.get(f"{base}_recent", np.nan)
            if not valid(a): a = np.nan
            if not valid(r): r = np.nan
            pct = pct_from_ratio_team(safe_ratio_team(r, a))
            cat_rows.append((label, pct, a, r))
        if cat_rows:
            cur = write_section(ws, cur, "CATAPULT", ["Metric", "% change", "Avg", "Recent", "Notes"], cat_rows, band=PCT_BAND)
        else:
            cur += 1

        fd_rows = []
        for base, label in forcedecks_metrics:
            a = rowp.get(f"{base}_avg", np.nan)
            r = rowp.get(f"{base}_recent", np.nan)
            if not valid(a): a = np.nan
            if not valid(r): r = np.nan
            pct = pct_from_ratio_team(safe_ratio_team(r, a))
            fd_rows.append((label, pct, a, r))
        if fd_rows:
            cur = write_section(ws, cur, "FORCEDECKS", ["Metric", "% change", "Avg", "Recent", "Notes"], fd_rows, band=PCT_BAND)
        else:
            cur += 1

        nb_rows = []
        for suf, label in NORDBORD_PAIRS:
            for side, sname in (("left", "Left"), ("right", "Right")):
                a_col = f"{side}{suf}_avg"
                r_col = f"{side}{suf}_recent"
                if a_col not in df.columns or r_col not in df.columns:
                    continue
                a = rowp.get(a_col, np.nan)
                r = rowp.get(r_col, np.nan)
                if not valid(a): a = np.nan
                if not valid(r): r = np.nan
                pct = pct_from_ratio_team(safe_ratio_team(r, a))
                nb_rows.append((f"{sname} {label}", pct, a, r))
        if nb_rows:
            cur = write_section(ws, cur, "NORDBORD", ["Metric", "% change", "Avg", "Recent", "Notes"], nb_rows, band=PCT_BAND)
        else:
            cur += 1

        ar_rows = []
        for suf, label in NORDBORD_PAIRS:
            lcol = f"left{suf}_recent"
            rcol = f"right{suf}_recent"
            if lcol not in df.columns or rcol not in df.columns:
                continue
            L = rowp.get(lcol, np.nan)
            R = rowp.get(rcol, np.nan)
            if not valid(L): L = np.nan
            if not valid(R): R = np.nan
            a = calc_asym_team(L, R)
            ar_rows.append((label, a, L, R))
        if ar_rows:
            cur = write_section(ws, cur, "NORDBORD ASYMMETRY (RECENT)", ["Metric", "Asym", "Left", "Right", "Notes"], ar_rows, band=ASYM_BAND)
        else:
            cur += 1

        aa_rows = []
        for suf, label in NORDBORD_PAIRS:
            lcol = f"left{suf}_avg"
            rcol = f"right{suf}_avg"
            if lcol not in df.columns or rcol not in df.columns:
                continue
            L = rowp.get(lcol, np.nan)
            R = rowp.get(rcol, np.nan)
            if not valid(L): L = np.nan
            if not valid(R): R = np.nan
            a = calc_asym_team(L, R)
            aa_rows.append((label, a, L, R))
        if aa_rows:
            cur = write_section(ws, cur, "NORDBORD ASYMMETRY (AVERAGE)", ["Metric", "Asym", "Left", "Right", "Notes"], aa_rows, band=ASYM_BAND)

        ws.freeze_panes = "A3"
        _apply_col_widths(ws, min_w=12, max_w=45, extra=2)

    wb.save(out_path)
    return out_path

if __name__ == "__main__":
    generated_ts = _make_generated_ts()

    if RUN_COMPLETE_DATA:
        build_complete_data_csv(
            output_csv=COMPLETE_CSV,
            excluded_metrics=EXCLUDED_METRICS,
            excluded_players=EXCLUDED_PLAYERS,
        )

    if RUN_METRIC_VISUALIZATIONS:
        generate_metric_visualizations(
            csv_path=COMPLETE_CSV,
            out_dir="reports",
            pdf_name="metric_visualizations.pdf",
            make_pdf=True,
            PCT_BAND=OUTLIER_THRESHOLD,
            ASYM_BAND=OUTLIER_THRESHOLD,
            TREAT_ZERO_AS_MISSING=True,
            MAX_PANELS_PER_GROUP=16,
            GRID_COLS=4,
            ADD_PER_METRIC_PAGES=True,
            generated_ts=generated_ts,
        )

    if RUN_PLAYER_VISUALIZATIONS:
        generate_player_visualizations(
            csv_path=COMPLETE_CSV,
            out_dir="reports",
            pdf_name="player_visualizations.pdf",
            PCT_HIGHLIGHT=OUTLIER_THRESHOLD,
            ASYM_FLAG_PCT=OUTLIER_THRESHOLD,
            TREAT_ZERO_AS_MISSING=True,
            generated_ts=generated_ts,
        )

    if RUN_METRIC_SPREADSHEETS:
        generate_metric_spreadsheets(
            csv_path=COMPLETE_CSV,
            out_dir="reports",
            xlsx_name="metric_spreadsheets.xlsx",
            PCT_BAND=OUTLIER_THRESHOLD,
            ASYM_BAND=OUTLIER_THRESHOLD,
            TREAT_ZERO_AS_MISSING=True,
            generated_ts=generated_ts,
        )

    if RUN_PLAYER_SPREADSHEETS:
        generate_player_spreadsheets(
            csv_path=COMPLETE_CSV,
            out_dir="reports",
            xlsx_name="player_spreadsheets.xlsx",
            PCT_BAND=OUTLIER_THRESHOLD,
            ASYM_BAND=OUTLIER_THRESHOLD,
            TREAT_ZERO_AS_MISSING=True,
            generated_ts=generated_ts,
        )
